# Preprocessing

> This module store preprocessing funtions. 


In [ ]:
#| default_exp preprocessing

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import logging

import os
import numpy as np
import pandas as pd

In [ ]:
#| export
logger = logging.getLogger(name='preprocessing')
logging.basicConfig(filename="example.log",filemode='w+' ,level=logging.DEBUG, force=True, format='[%(asctime)-15s] %(levelname)-8s %(filename)s %(funcName)s line %(lineno)d %(message)s')

In [ ]:
#| export
def read_nodes(geometry # .csv or .key file containing source template
               ) -> np.ndarray: # Numpy array of shape [n_nodes, x,y,z_displacement]
    "Read the nodes from the source template. The source template can be either a .key/.k file or .csv"
    try:
        if geometry.endswith('.csv'):
            pass
        elif geometry.endswith('.key') or geometry.endswith('.k') :
            pass
    except:
        logger.exception("No readable file")
    else:
        return 

In [ ]:
#| export
def read_landmarks(file # .csv or .key file containing Landmarks
                   )-> pd.DataFrame: # Dataframe of length [n_landmarks] divided in columns [ID/label, x,y,z]

    "Read the landmarks from file."
    try:
        if file.endswith('.csv'):
            pass
        elif file.endswith('.key') or file.endswith('.k') :
            pass
    except:
        logger.exception("No readable file")
    else:
        return 
    pass

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()